In [1]:
# 1. Load Data
import pandas as pd

data = pd.read_csv('../DATA/raw_2023051820231018_경대기업맞춤형.csv')
data.head()

,time,E_scr_pv,E_scr_sv,c_temp_pv,c_temp_sv,k_rpm_pv,k_rpm_sv,n_temp_pv,n_temp_sv,scale_pv,s_temp_pv,s_temp_sv,Unnamed: 12
0,2023-05-18T15:09:19.018900Z,8,8,69.6,70,189,180,67.2,70,3.01,67.1,70,NaN
1,2023-05-18T15:09:20.128150Z,8,8,69.8,70,189,180,67.2,70,3.01,67.0,70,NaN
2,2023-05-18T15:12:38.819460Z,8,8,69.7,70,189,180,67.9,70,3.08,65.9,70,NaN
3,2023-05-18T15:12:39.958188Z,8,8,69.7,70,189,180,67.8,70,3.08,65.9,70,NaN
4,2023-05-18T15:12:41.050178Z,8,8,69.7,70,189,180,67.8,70,3.08,65.9,70,NaN


In [4]:
data.info() # Unnamed: 12 column 삭제, time -> datetime으로 변환
data = data.drop('Unnamed: 12', axis=1)
data['time'] = pd.to_datetime(data['time'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235413 entries, 0 to 235412
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   time         235413 non-null  object 
 1   E_scr_pv     235413 non-null  int64  
 2   E_scr_sv     235413 non-null  int64  
 3   c_temp_pv    235413 non-null  float64
 4   c_temp_sv    235413 non-null  int64  
 5   k_rpm_pv     235413 non-null  int64  
 6   k_rpm_sv     235413 non-null  int64  
 7   n_temp_pv    235413 non-null  float64
 8   n_temp_sv    235413 non-null  int64  
 9   scale_pv     235413 non-null  float64
 10  s_temp_pv    235413 non-null  float64
 11  s_temp_sv    235413 non-null  int64  
 12  Unnamed: 12  204865 non-null  float64
dtypes: float64(5), int64(7), object(1)
memory usage: 23.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235413 entries, 0 to 235412
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     -

In [5]:
data.describe()

,E_scr_pv,E_scr_sv,c_temp_pv,c_temp_sv,k_rpm_pv,k_rpm_sv,n_temp_pv,n_temp_sv,scale_pv,s_temp_pv,s_temp_sv
count,235413.000000,235413.000000,235413.000000,235413.0,235413.000000,235413.000000,235413.000000,235413.000000,235413.000000,235413.000000,235413.0
mean,7.891026,7.998581,70.211779,70.0,178.204662,609.621346,67.841010,69.985727,5.257169,67.637708,70.0
std,0.924159,0.074126,0.524797,0.0,22.461606,735.412105,1.119829,0.999448,55.582926,1.276029,0.0
min,0.000000,7.000000,65.100000,70.0,0.000000,130.000000,57.700000,0.000000,0.000000,65.400000,70.0
25%,8.000000,8.000000,69.800000,70.0,176.000000,180.000000,67.000000,70.000000,0.000000,66.600000,70.0
50%,8.000000,8.000000,70.200000,70.0,181.000000,180.000000,67.400000,70.000000,0.000000,67.500000,70.0
75%,8.000000,8.000000,70.500000,70.0,187.000000,1730.000000,68.600000,70.000000,0.000000,68.400000,70.0
max,12.000000,12.000000,72.800000,70.0,203.000000,2030.000000,71.200000,70.000000,655.350000,72.400000,70.0


In [7]:
# 3. 1차 데이터와 동일한 전처리 가능한지 분석해보기

# - _sv로 끝나는 컬럼명 value_counts() 확인
for col in data.columns:
    if col.endswith('_sv'):
        print(col, data[col].value_counts())
        
# [해석]
# 1. temp sv 는 n_temp_sv == 0 인 48개 외 모두 하나의 값 -> sv 컬럼 삭제 가능
# 2. rpm 후반부는 1000이상이면 1/10, sv와 pv가 동일하므로 sv 컬럼 삭제 가능

# - 컬럼 삭제
data = data.drop(['c_temp_sv', 'n_temp_sv', 's_temp_sv', 'k_rpm_sv'], axis=1)

E_scr_sv E_scr_sv
8     234839
7        526
12        48
Name: count, dtype: int64
c_temp_sv c_temp_sv
70    235413
Name: count, dtype: int64
k_rpm_sv k_rpm_sv
180     144004
150      28162
1880     12699
1870      8350
1890      5802
         ...  
1610         9
1470         3
1530         2
1990         1
2020         1
Name: count, Length: 61, dtype: int64
n_temp_sv n_temp_sv
70    235365
0         48
Name: count, dtype: int64
s_temp_sv s_temp_sv
70    235413
Name: count, dtype: int64


In [10]:
# E_scr_sv = 12 인 행 확인
data[data['E_scr_sv'] == 12].describe()    # scale_pv 평균 3.93, 대표적인 오차값; 오류로 보긴 어려움

,E_scr_pv,E_scr_sv,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
count,48.0,48.0,48.000000,48.00000,48.000000,48.000000,48.000000
mean,12.0,12.0,71.150000,141.56250,59.970833,3.938125,68.447917
std,0.0,0.0,1.058903,10.35314,2.923053,0.147066,1.190206
min,12.0,12.0,70.100000,117.00000,57.700000,3.590000,67.100000
25%,12.0,12.0,70.200000,140.00000,57.900000,3.850000,67.475000
50%,12.0,12.0,70.500000,147.50000,58.000000,4.030000,68.000000
75%,12.0,12.0,72.700000,150.00000,64.300000,4.040000,69.900000
max,12.0,12.0,72.800000,150.00000,64.800000,4.070000,70.600000


In [11]:
# 2 < scale_pv < 5 인 행 확인
data[(data['scale_pv'] > 2) & (data['scale_pv'] < 5)].describe()    # 전체 23만개, 여긴 3만8천개

,E_scr_pv,E_scr_sv,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
count,38719.000000,38719.000000,38719.000000,38719.000000,38719.000000,38719.000000,38719.000000
mean,7.987500,7.991425,70.255748,180.966554,67.566970,3.042828,67.381237
std,0.254204,0.182472,0.507793,8.711707,0.941835,0.050211,1.117110
min,0.000000,7.000000,68.600000,0.000000,57.700000,2.560000,65.400000
25%,8.000000,8.000000,70.000000,176.000000,67.000000,3.020000,66.400000
50%,8.000000,8.000000,70.300000,182.000000,67.300000,3.040000,67.300000
75%,8.000000,8.000000,70.600000,187.000000,67.900000,3.060000,68.100000
max,12.000000,12.000000,72.800000,202.000000,71.100000,4.070000,71.500000


In [12]:
# - scale_pv 2 미만, 5 이상인 행 확인
data[(data['scale_pv'] < 2) | (data['scale_pv'] >= 5)].describe()

,E_scr_pv,E_scr_sv,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
count,196693.000000,196693.000000,196693.000000,196693.000000,196693.000000,196693.000000,196693.000000
mean,7.872034,7.999990,70.203123,177.660944,67.894957,5.693078,67.688192
std,1.003636,0.003189,0.527650,24.230257,1.143927,60.798725,1.299089
min,0.000000,7.000000,65.100000,0.000000,65.800000,0.000000,65.400000
25%,8.000000,8.000000,69.800000,176.000000,67.000000,0.000000,66.600000
50%,8.000000,8.000000,70.200000,181.000000,67.500000,0.000000,67.600000
75%,8.000000,8.000000,70.500000,187.000000,68.800000,0.000000,68.400000
max,8.000000,8.000000,71.900000,203.000000,71.200000,655.350000,72.400000


### 데이터 범위가 다르다 : scale을 더 러프하게 잡고 학습과 평가 데이터셋을 나누기 (10월)

In [15]:
data = pd.read_csv('../DATA/raw_2023051820231018_경대기업맞춤형.csv')

# time의 월이 10월인 행 확인
oct_data = data[data['time'].str.contains('-10-', regex=True)]
oct_data.describe()

train_data = data[data['time'] < '2023-10-01']
test_data = data[data['time'] >= '2023-10-01']
print(train_data.shape, test_data.shape)

(205736, 13) (29677, 13)


In [16]:
# 4. 전처리 후 데이터 저장
# - Unnamed: 12 컬럼 삭제
data = data.drop('Unnamed: 12', axis=1)

# - 결측치 확인
print('결측치 개수 목록\n', data.isnull().sum())

# - 중복 데이터 확인
print('중복 데이터 개수 :', data.duplicated().sum())

결측치 개수 목록
 time         0
E_scr_pv     0
E_scr_sv     0
c_temp_pv    0
c_temp_sv    0
k_rpm_pv     0
k_rpm_sv     0
n_temp_pv    0
n_temp_sv    0
scale_pv     0
s_temp_pv    0
s_temp_sv    0
dtype: int64
중복 데이터 개수 : 0


In [19]:
# scale_pv < 5 인 행 확인
# - 0 값은 보간 처리
data[data['scale_pv'] < 5].describe()

,E_scr_pv,E_scr_sv,c_temp_pv,c_temp_sv,k_rpm_pv,k_rpm_sv,n_temp_pv,n_temp_sv,scale_pv,s_temp_pv,s_temp_sv
count,233676.00000,233676.000000,233676.000000,233676.0,233676.000000,233676.000000,233676.000000,233676.000000,233676.000000,233676.000000,233676.0
mean,7.89179,7.998571,70.212484,70.0,178.250064,613.034629,67.828551,69.985621,0.505085,67.626146,70.0
std,0.92091,0.074401,0.526108,0.0,22.404608,737.069949,1.110944,1.003155,1.131264,1.269115,0.0
min,0.00000,7.000000,65.100000,70.0,0.000000,130.000000,57.700000,0.000000,0.000000,65.400000,70.0
25%,8.00000,8.000000,69.800000,70.0,176.000000,180.000000,67.000000,70.000000,0.000000,66.600000,70.0
50%,8.00000,8.000000,70.200000,70.0,181.000000,180.000000,67.400000,70.000000,0.000000,67.500000,70.0
75%,8.00000,8.000000,70.500000,70.0,187.000000,1740.000000,68.600000,70.000000,0.000000,68.400000,70.0
max,12.00000,12.000000,72.800000,70.0,203.000000,2030.000000,71.200000,70.000000,4.070000,72.400000,70.0


In [ ]:
data = data[data['scale_pv'] < 5]
